# Data postprocessing

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import IPython

dataset_name = 'parser_dataset.csv'
postprocessed_dataset_name = 'postprocessed_dataset.csv'
processed_dataset_name = 'processed_dataset.csv'

pictures_folder = 'Pictures'

In [ ]:
columns_list = ['Name', 'Code', 'Category']

try:
    df = pd.read_csv(dataset_name)
except Exception: 
    df = pd.DataFrame(columns = columns_list)
    
assert (df.columns == columns_list).all()

df.info()

In [ ]:
def extract_scopes(row):
#     print(row)
    name = row[0]
    code = row[1]

    idx = 0
    bracket_idx = 0
    open_bracket_pos = [None]
    
    scopes = list()
    scope_processed = None
    for_processed = None
    
    def check(key):
        prevIdx = idx - 1
        postIdx = idx + len(key)

        if code[idx : postIdx] != key:
            return False

        return (prevIdx == -1 or not code[prevIdx].isalnum()) and \
               (postIdx >= len(code) or not code[postIdx].isalnum())

    
    while 0 <= idx < len(code):
        if idx < len(code) - 1 and code[idx:idx+2] == r'/*':
            idx = code.find(r'*/', idx)
        if idx < len(code) - 1 and code[idx:idx+2] == r'//':
            idx = code.find('\n', idx)
        if code[idx] == r'"':
            idx = code.find(r'"', idx + 1);
        
        if idx == -1:
            continue
            
        if code[idx] == '{':
            bracket_idx += 1
            
            if bracket_idx < len(open_bracket_pos):
                open_bracket_pos[bracket_idx] = idx
            else:
                open_bracket_pos.append(idx)
            
            if for_processed is not None:
                open_bracket_pos[bracket_idx] = for_processed
                for_processed = None
                
        elif code[idx] == '}':
            if scope_processed == bracket_idx:
#                 print(bracket_idx, scope_processed)
#                 print(open_bracket_pos)
                assert open_bracket_pos[scope_processed] is not None
                scopes.append(code[open_bracket_pos[scope_processed] : idx+1])
                scope_processed = None
            
            open_bracket_pos[bracket_idx] = None
            bracket_idx -= 1
            
        elif scope_processed is None and check('for'):
            for_processed = idx
            
            idx = code.find('(', idx) + 1
            bra_idx = 1
            is_name = False
            
            while bra_idx != 0:
                if code[idx:idx+2] == r'/*':
                    idx = code.find(r'*/', idx)
                if code[idx:idx+2] == r'//':
                    idx = code.find('\n', idx)
                if code[idx] == r'"':
                    idx = code.find(r'"', idx + 1);
                
                if code[idx] == '(':
                    bra_idx += 1
                    
                elif code[idx] == ')':
                    bra_idx -= 1

                elif not is_name and check(name):
                    is_name = True
                
                idx += 1
            
            if is_name:
                while code[idx] != '{' and code[idx] != ';':
                    idx += 1

                if code[idx] == ';':
                    scopes.append(code[for_processed : idx+1])
                    for_processed = None
                else:
                    scope_processed = bracket_idx+1
                    continue
            
        
        elif scope_processed is None and check(name):
            scope_processed = bracket_idx
        
        idx += 1
        
#     print("------------Scopes---------------")
#     for scope in scopes:
#         print(scope)
#         print('-'*20)
    
    elems = len(scopes)
    return {'Name': [name]*elems,
            'Code': scopes,
            'Category': [row[2]]*elems}
    

In [ ]:
some_shit = [extract_scopes(x) for x in df[columns_list].to_numpy()]
# some_shit = extract_scopes(df.iloc[17].to_numpy())

In [ ]:
parsed_dict = {name: [] for name in columns_list}
for d in some_shit:
    for name in columns_list:
        parsed_dict[name] += d[name] 

In [ ]:
parsed_df = pd.DataFrame(parsed_dict)
parsed_df.head(30)

In [ ]:
parsed_df.info()

If you want to validate all the examples and drop extra ones / change a name, run the code below:

In [ ]:
i = 0

In [ ]:
while i < len(parsed_dict['Name']):
    print(f'Check of sample {i}')
    print('-'*20)
    print(f'Variable name: {parsed_dict["Name"][i]}')
    print(parsed_dict['Code'][i])
    print('-'*20)
    
    inp = input()
    if inp == '$d':
        for cols in columns_list:
            del parsed_dict[cols][i]
        
    elif inp != '':
        parsed_dict['Name'][i] = inp
        
    IPython.display.clear_output()
    i += 1

In [ ]:
parsed_df = pd.DataFrame(parsed_dict)
parsed_df.info()

Saving chagnes to the file

In [ ]:
parsed_df.to_csv(postprocessed_dataset_name, index = False)

In [14]:
with open('file_progress.txt', 'r+') as f:
    text = '\n'.join(f.read().split('$'))
    print(text)
    f.write(text)

./reps/Arduino/arduino-core/src/processing/app/Sketch.java
./reps/java-design-patterns/facade/src/main/java/com/iluwatar/facade/DwarvenTunnelDigger.java
./reps/java-design-patterns/throttling/src/main/java/com/iluwatar/throttling/Bartender.java
./reps/RxJava/src/test/java/io/reactivex/rxjava3/internal/operators/maybe/MaybeTimeIntervalTest.java
./reps/java-design-patterns/decorator/src/main/java/com/iluwatar/decorator/ClubbedTroll.java
./reps/java-design-patterns/composite/src/main/java/com/iluwatar/composite/Sentence.java
./reps/java-design-patterns/metadata-mapping/src/test/java/com/iluwatar/metamapping/AppTest.java
./reps/java-design-patterns/master-worker-pattern/src/main/java/com/iluwatar/masterworker/system/ArrayTransposeMasterWorker.java
./reps/RxJava/src/test/java/io/reactivex/rxjava3/internal/operators/flowable/FlowableAsObservableTest.java
./reps/java-design-patterns/separated-interface/src/test/java/com/iluwatar/separatedinterface/taxes/ForeignTaxCalculatorTest.java
./reps/ja